# GraphQL

GraphQL é uma linguagem de consulta e runtime para APIs desenvolvida pelo Facebook em 2012 e open-sourced em 2015. É uma alternativa ao REST que oferece uma abordagem mais flexível e eficiente para buscar e manipular dados.

**Principais Características**:

- **Query Language**: Permite que clientes especifiquem exatamente quais dados precisam
- **Tipo Seguro**: Sistema de tipos forte que define a estrutura da API
- **Single Endpoint**: Uma única URL para todas as operações
- **Introspective**: A API pode descrever sua própria estrutura
- **Hierarchical**: Segue a estrutura hierárquica dos dados


## Conceitos Fundamentais

### 1. Schema
O schema define a estrutura da API, incluindo tipos, queries, mutations e subscriptions.

### 2. Types (Tipos)
- **Scalar Types**: String, Int, Float, Boolean, ID
- **Object Types**: Tipos customizados
- **Enum Types**: Conjunto de valores permitidos
- **Interface Types**: Contratos que tipos devem implementar
- **Union Types**: Tipos que podem ser um de vários tipos

### 3. Operations (Operações)
- **Query**: Leitura de dados
- **Mutation**: Modificação de dados
- **Subscription**: Dados em tempo real

### 4. Resolvers
Funções que buscam os dados para cada campo do schema.


## GraphQL vs REST

| Aspecto        | REST                  | GraphQL             |
| -------------- | --------------------- | ------------------- |
| Endpoints      | Múltiplos             | Único               |
| Over-fetching  | Problema comum        | Evitado             |
| Under-fetching | Múltiplas requisições | Uma requisição      |
| Versionamento  | URLs versionadas      | Evolution do schema |
| Caching        | HTTP caching          | Mais complexo       |
| Aprendizado    | Mais simples          | Curva maior         |


## Instalação e Setup

### Bibliotecas Principais

- Para servidor GraphQL
  - graphene
  - graphene-django  # Para Django
  - strawberry-graphql  # Alternativa moderna
  - ariadne  # Schema-first

- Para cliente GraphQL
  -gql
  -sgqlc
  -requests  # Para requisições HTTP simples

In [3]:
%pip install graphene graphene-django strawberry-graphql ariadne
%pip install gql sgqlc requests

  DEPRECATION: Building 'promise' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'promise'. Discussion can be found at https://github.com/pypa/pip/issues/6334



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21547 sha256=4d027b8892a8e8e883828f9d78184423855f79d1221c4099d145ad38ec8c6971
  Stored in directory: c:\users\helbe\appdata\local\pip\cache\wheels\54\4e\28\3ed0e1c8a752867445bab994d2340724928aa3ab059c57c8db
Successfully built promise

   ------ --------------------------------- 1/6 [promise]
  Attempting uninstall: graphql-core
   ------ --------------------------------- 1/6 [promise]
    Found existing installation: graphql-core 3.2.6
   ------ --------------------------------- 1/6 [promise]
   ------------- -------------------------- 2/6 [graphql-core]
    Uninstalling graphql-core-3.2.6:
   ------------- -------------------------- 2/6 [graphql-core]
      Successfully uninstalled graphql-core-3.2.6
   ------------- -------------------------- 2/6 [graphql-core]
   ------------- -------------------------- 2/6 [gr

## Exemplo Prático: Biblioteca com Graphene

### 1. Definindo o Schema

In [4]:
import graphene
from graphene import ObjectType, String, Int, List, Boolean, Field, Mutation, Schema
from datetime import datetime

In [5]:
# Tipos de dados
class Author(ObjectType):
    id = Int()
    name = String()
    email = String()
    books = List(lambda: Book)
    
    def resolve_books(self, info):
        return [book for book in books_data if book['author_id'] == self.id]

class Book(ObjectType):
    id = Int()
    title = String()
    author_id = Int()
    author = Field(Author)
    year = Int()
    available = Boolean()
    
    def resolve_author(self, info):
        return next((author for author in authors_data if author['id'] == self.author_id), None)

# Dados de exemplo
authors_data = [
    {'id': 1, 'name': 'George Orwell', 'email': 'george@example.com'},
    {'id': 2, 'name': 'J.K. Rowling', 'email': 'jk@example.com'},
    {'id': 3, 'name': 'Isaac Asimov', 'email': 'isaac@example.com'}
]

books_data = [
    {'id': 1, 'title': '1984', 'author_id': 1, 'year': 1949, 'available': True},
    {'id': 2, 'title': 'Animal Farm', 'author_id': 1, 'year': 1945, 'available': True},
    {'id': 3, 'title': 'Harry Potter', 'author_id': 2, 'year': 1997, 'available': False},
    {'id': 4, 'title': 'Foundation', 'author_id': 3, 'year': 1951, 'available': True}
]

# Queries
class Query(ObjectType):
    # Buscar todos os livros
    books = List(Book)
    
    # Buscar livro por ID
    book = Field(Book, id=Int(required=True))
    
    # Buscar todos os autores
    authors = List(Author)
    
    # Buscar autor por ID
    author = Field(Author, id=Int(required=True))
    
    # Buscar livros disponíveis
    available_books = List(Book)
    
    # Buscar livros por ano
    books_by_year = List(Book, year=Int())
    
    def resolve_books(self, info):
        return books_data
    
    def resolve_book(self, info, id):
        return next((book for book in books_data if book['id'] == id), None)
    
    def resolve_authors(self, info):
        return authors_data
    
    def resolve_author(self, info, id):
        return next((author for author in authors_data if author['id'] == id), None)
    
    def resolve_available_books(self, info):
        return [book for book in books_data if book['available']]
    
    def resolve_books_by_year(self, info, year=None):
        if year:
            return [book for book in books_data if book['year'] == year]
        return books_data

# Mutations
class CreateBook(Mutation):
    class Arguments:
        title = String(required=True)
        author_id = Int(required=True)
        year = Int()
        available = Boolean()
    
    book = Field(Book)
    
    def mutate(self, info, title, author_id, year=None, available=True):
        new_id = max([book['id'] for book in books_data]) + 1
        new_book = {
            'id': new_id,
            'title': title,
            'author_id': author_id,
            'year': year or datetime.now().year,
            'available': available
        }
        books_data.append(new_book)
        return CreateBook(book=new_book)

class UpdateBook(Mutation):
    class Arguments:
        id = Int(required=True)
        title = String()
        available = Boolean()
    
    book = Field(Book)
    
    def mutate(self, info, id, title=None, available=None):
        book = next((book for book in books_data if book['id'] == id), None)
        if not book:
            raise Exception(f"Livro com ID {id} não encontrado")
        
        if title:
            book['title'] = title
        if available is not None:
            book['available'] = available
            
        return UpdateBook(book=book)

class DeleteBook(Mutation):
    class Arguments:
        id = Int(required=True)
    
    success = Boolean()
    
    def mutate(self, info, id):
        global books_data
        original_length = len(books_data)
        books_data = [book for book in books_data if book['id'] != id]
        return DeleteBook(success=len(books_data) < original_length)

class Mutations(ObjectType):
    create_book = CreateBook.Field()
    update_book = UpdateBook.Field()
    delete_book = DeleteBook.Field()

# Schema principal
schema = Schema(query=Query, mutation=Mutations)

### 2. Servidor GraphQL com Flask


In [7]:
from flask import Flask
from flask_graphql import GraphQLView

app = Flask(__name__)

# Endpoint GraphQL
app.add_url_rule(
    '/graphql',
    view_func=GraphQLView.as_view('graphql', schema=schema, graphiql=True)
)

# Endpoint GraphiQL (interface web)
app.add_url_rule(
    '/graphiql',
    view_func=GraphQLView.as_view('graphiql', schema=schema, graphiql=True)
)

if __name__ == '__main__':
    app.run(debug=True)

ModuleNotFoundError: No module named 'flask_graphql'

### 3. Cliente GraphQL


In [6]:
import requests
import json
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

class GraphQLClient:
    def __init__(self, url):
        self.url = url
        self.transport = RequestsHTTPTransport(url=url)
        self.client = Client(transport=self.transport, fetch_schema_from_transport=True)
    
    def execute_query(self, query, variables=None):
        """Executa uma query GraphQL"""
        try:
            document = gql(query)
            return self.client.execute(document, variable_values=variables)
        except Exception as e:
            print(f"Erro na query: {e}")
            return None
    
    def simple_request(self, query, variables=None):
        """Requisição HTTP simples"""
        payload = {
            'query': query,
            'variables': variables or {}
        }
        
        response = requests.post(
            self.url,
            json=payload,
            headers={'Content-Type': 'application/json'}
        )
        
        return response.json()

# Exemplo de uso
if __name__ == "__main__":
    client = GraphQLClient("http://localhost:5000/graphql")
    
    # Query 1: Buscar todos os livros com autores
    query1 = """
    query GetBooksWithAuthors {
        books {
            id
            title
            year
            available
            author {
                name
                email
            }
        }
    }
    """
    
    result1 = client.execute_query(query1)
    print("Livros com autores:")
    print(json.dumps(result1, indent=2))
    
    # Query 2: Buscar livro específico
    query2 = """
    query GetBook($bookId: Int!) {
        book(id: $bookId) {
            title
            year
            author {
                name
            }
        }
    }
    """
    
    result2 = client.execute_query(query2, {"bookId": 1})
    print("\nLivro específico:")
    print(json.dumps(result2, indent=2))
    
    # Mutation: Criar novo livro
    mutation1 = """
    mutation CreateNewBook($title: String!, $authorId: Int!, $year: Int) {
        createBook(title: $title, authorId: $authorId, year: $year) {
            book {
                id
                title
                year
                author {
                    name
                }
            }
        }
    }
    """
    
    result3 = client.execute_query(mutation1, {
        "title": "Brave New World",
        "authorId": 1,
        "year": 1932
    })
    print("\nNovo livro criado:")
    print(json.dumps(result3, indent=2))

Erro na query: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /graphql (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246265CEE30>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))
Livros com autores:
null
Erro na query: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /graphql (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246265CED10>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))

Livro específico:
null
Erro na query: HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /graphql (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246265CF730>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão

## Exemplos de Queries

### Queries Básicas


```graphql
# Buscar todos os livros
query {
    books {
        id
        title
        year
    }
}

# Buscar livro por ID
query {
    book(id: 1) {
        title
        author {
            name
        }
    }
}

# Buscar apenas livros disponíveis
query {
    availableBooks {
        title
        year
    }
}
```

### Queries com Variáveis

```graphql
query GetBooksByYear($year: Int!) {
    booksByYear(year: $year) {
        title
        author {
            name
        }
    }
}
```

### Mutations

```graphql
# Criar livro
mutation {
    createBook(title: "New Book", authorId: 1, year: 2023) {
        book {
            id
            title
        }
    }
}

# Atualizar livro
mutation {
    updateBook(id: 1, available: false) {
        book {
            title
            available
        }
    }
}

# Deletar livro
mutation {
    deleteBook(id: 1) {
        success
    }
}
```

## Exemplo com Strawberry (Alternativa Moderna)


In [ ]:
import strawberry
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class AuthorData:
    id: int
    name: str
    email: str

@dataclass
class BookData:
    id: int
    title: str
    author_id: int
    year: int
    available: bool

# Tipos GraphQL
@strawberry.type
class Author:
    id: int
    name: str
    email: str
    
    @strawberry.field
    def books(self) -> List['Book']:
        return [Book(**book.__dict__) for book in books_db if book.author_id == self.id]

@strawberry.type
class Book:
    id: int
    title: str
    author_id: int
    year: int
    available: bool
    
    @strawberry.field
    def author(self) -> Optional[Author]:
        author_data = next((a for a in authors_db if a.id == self.author_id), None)
        return Author(**author_data.__dict__) if author_data else None

# Dados
authors_db = [
    AuthorData(1, "George Orwell", "george@example.com"),
    AuthorData(2, "J.K. Rowling", "jk@example.com"),
]

books_db = [
    BookData(1, "1984", 1, 1949, True),
    BookData(2, "Harry Potter", 2, 1997, False),
]

# Queries
@strawberry.type
class Query:
    @strawberry.field
    def books(self) -> List[Book]:
        return [Book(**book.__dict__) for book in books_db]
    
    @strawberry.field
    def book(self, id: int) -> Optional[Book]:
        book_data = next((b for b in books_db if b.id == id), None)
        return Book(**book_data.__dict__) if book_data else None

# Mutations
@strawberry.input
class BookInput:
    title: str
    author_id: int
    year: Optional[int] = None
    available: bool = True

@strawberry.type
class Mutation:
    @strawberry.mutation
    def create_book(self, book_input: BookInput) -> Book:
        new_id = max([b.id for b in books_db]) + 1
        new_book = BookData(
            id=new_id,
            title=book_input.title,
            author_id=book_input.author_id,
            year=book_input.year or 2023,
            available=book_input.available
        )
        books_db.append(new_book)
        return Book(**new_book.__dict__)

# Schema
schema = strawberry.Schema(query=Query, mutation=Mutation)

# Executar
if __name__ == "__main__":
    query = """
    query {
        books {
            title
            author {
                name
            }
        }
    }
    """
    
    result = schema.execute_sync(query)
    print(result.data)

## Integração com Banco de Dados

### Exemplo com SQLAlchemy


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import graphene
from graphene_sqlalchemy import SQLAlchemyObjectType, SQLAlchemyConnectionField

# Configuração do banco
engine = create_engine('sqlite:///library.db')
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Modelos SQLAlchemy
class AuthorModel(Base):
    __tablename__ = 'authors'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    books = relationship("BookModel", back_populates="author")

class BookModel(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    author_id = Column(Integer, ForeignKey('authors.id'))
    year = Column(Integer)
    available = Column(Boolean)
    author = relationship("AuthorModel", back_populates="books")

# Criar tabelas
Base.metadata.create_all(engine)

# Tipos GraphQL
class Author(SQLAlchemyObjectType):
    class Meta:
        model = AuthorModel

class Book(SQLAlchemyObjectType):
    class Meta:
        model = BookModel

# Query
class Query(graphene.ObjectType):
    books = graphene.List(Book)
    authors = graphene.List(Author)
    
    def resolve_books(self, info):
        session = Session()
        return session.query(BookModel).all()
    
    def resolve_authors(self, info):
        session = Session()
        return session.query(AuthorModel).all()

schema = graphene.Schema(query=Query)

## Ferramentas e Utilitários

### GraphQL Playground Local

In [8]:
from graphql_server.flask import GraphQLView
from flask import Flask
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

app.add_url_rule(
    '/graphql',
    view_func=GraphQLView.as_view(
        'graphql',
        schema=schema,
        graphiql=True,  # Interface web habilitada
        pretty=True
    )
)

if __name__ == '__main__':
    print("GraphQL Playground disponível em: http://localhost:5000/graphql")
    app.run(debug=True)

ModuleNotFoundError: No module named 'graphql_server'

### Cliente para Testes


In [ ]:
import asyncio
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

class AsyncGraphQLClient:
    def __init__(self, url):
        self.transport = AIOHTTPTransport(url=url)
        self.client = Client(transport=self.transport)
    
    async def execute(self, query, variables=None):
        async with self.client as session:
            result = await session.execute(gql(query), variable_values=variables)
            return result

# Exemplo de uso assíncrono
async def main():
    client = AsyncGraphQLClient("http://localhost:5000/graphql")
    
    query = """
    query {
        books {
            title
            author {
                name
            }
        }
    }
    """
    
    result = await client.execute(query)
    print(result)

# Executar
# asyncio.run(main())

## Boas Práticas

### 1. Estrutura de Projeto

```text
projeto/
├── schema/
│   ├── __init__.py
│   ├── types.py
│   ├── queries.py
│   ├── mutations.py
│   └── subscriptions.py
├── models/
│   ├── __init__.py
│   └── database.py
├── resolvers/
│   ├── __init__.py
│   └── book_resolvers.py
├── app.py
└── requirements.txt
```

### 2. Tratamento de Erros


In [ ]:
import graphene
from graphene import ObjectType, String, Field, Int

class BookNotFoundError(Exception):
    pass

class ErrorType(ObjectType):
    message = String()
    code = String()

class BookResult(ObjectType):
    book = Field(Book)
    error = Field(ErrorType)

class Query(ObjectType):
    get_book = Field(BookResult, id=Int(required=True))
    
    def resolve_get_book(self, info, id):
        try:
            book = get_book_by_id(id)
            if not book:
                return BookResult(error=ErrorType(message="Livro não encontrado", code="NOT_FOUND"))
            return BookResult(book=book)
        except Exception as e:
            return BookResult(error=ErrorType(message=str(e), code="INTERNAL_ERROR"))

### 3. Autenticação e Autorização


In [ ]:
from functools import wraps
import jwt

def login_required(f):
    @wraps(f)
    def decorated_function(self, info, *args, **kwargs):
        token = info.context.get('HTTP_AUTHORIZATION')
        if not token:
            raise Exception("Token de autenticação necessário")
        
        try:
            payload = jwt.decode(token, 'secret', algorithms=['HS256'])
            info.context['user'] = payload
        except jwt.InvalidTokenError:
            raise Exception("Token inválido")
        
        return f(self, info, *args, **kwargs)
    return decorated_function

class Query(ObjectType):
    @login_required
    def resolve_private_books(self, info):
        user = info.context['user']
        return get_user_books(user['id'])

### 4. Paginação


In [ ]:
import graphene
from graphene import ObjectType, String, Int, List, Field

class PageInfo(ObjectType):
    has_next_page = graphene.Boolean()
    has_previous_page = graphene.Boolean()
    start_cursor = graphene.String()
    end_cursor = graphene.String()

class BookConnection(ObjectType):
    edges = List(Book)
    page_info = Field(PageInfo)
    total_count = Int()

class Query(ObjectType):
    books = Field(
        BookConnection,
        first=Int(),
        after=String(),
        last=Int(),
        before=String()
    )
    
    def resolve_books(self, info, first=None, after=None, last=None, before=None):
        # Implementar lógica de paginação
        books = get_paginated_books(first, after, last, before)
        return BookConnection(
            edges=books,
            page_info=PageInfo(
                has_next_page=has_next_page(books),
                has_previous_page=has_previous_page(books)
            ),
            total_count=get_total_books_count()
        )

## Vantagens e Desvantagens

### Vantagens
- **Flexibilidade**: Cliente especifica exatamente os dados necessários
- **Eficiência**: Reduz over-fetching e under-fetching
- **Tipagem Forte**: Validação automática e documentação
- **Evolução**: Schema pode evoluir sem versioning
- **Ferramentas**: Excelente ecossistema de ferramentas

### Desvantagens
- **Curva de Aprendizado**: Mais complexo que REST
- **Caching**: Mais difícil de implementar
- **Complexidade**: Queries complexas podem ser custosas
- **Overhead**: Pode ser excessivo para APIs simples
